# Roocs data stores

Roocs consists of stores which it uses to identify and apply fixes to datasets. This notebook demonstrates the purpose of and how to interact with the Character store and the Fix store.

The Character store contains information about the 'character' of datasets that have been scanned. This is used in the identification of fixes.
    
The Fix store keeps track of these fixes. Each document corresponds to a specific dataset and details the fixes which are to be applied to it before an operation is applied to it.

In [ ]:
DIAGRAM

## Working with the character store

In [3]:
from .dachar.utils import get_stores
char_store = get_stores.get_dc_store('elasticsearch')
char_store.get_all()

ModuleNotFoundError: No module named '__main__.dachar'; '__main__' is not a package